In [1]:
import spacy
import pandas as pd
import en_core_web_md

In [2]:
nlp = en_core_web_md.load()
df = pd.read_csv('../Word2Vec/globe_data/bostonglobe2018.csv')

In [3]:
# removing special characters
def remove_specChar(df):

    spec_chars = ["!",'"',"#","%","&","'","(",")", "*","+",",",
                  "-",".","/",":",";","<", "=",">","?","@","[",
                  "\\","]","^","_", "`","{","|","}","~","–", 
                  "\xc2", "\xa0", "\x80", "\x9c", "\x99", "\x94", 
                  "\xad", "\xe2", "\x9d", "\n"]

    for char in spec_chars:
        df['text'] = df['text'].str.strip()
        df['text'] = df['text'].str.replace(char, ' ')
        
    return df

In [4]:
df = remove_specChar(df)
print("shape: ", df.shape)

shape:  (8291, 1)


In [5]:
doc = nlp(df.iloc[1]['text'])
len(doc)

2119

In [6]:
ents = []
for ent in doc.ents:
    if ent.label_ == 'PERSON' or ent.label_ == 'ORG':
        ents.append((ent.text, ent.label_, ent.root.head, ''))
        

In [7]:
people = []
for ent in ents:
    if ent[1] == 'PERSON':
        name = ent[0].split()
        if name != []:
            people.append(name[-1])

In [8]:
len(people)

142

In [9]:
names_df = pd.DataFrame(people)
names_df.columns = ['name']
names_df.head()

,name
0,Berry
1,Moore
2,—
3,Law
4,Law


In [10]:
from ethnicolr import census_ln, pred_census_ln

race_pred = pred_census_ln(names_df, 'name', 2010)

Using TensorFlow backend.
C:\Users\ASUS\anaconda3\envs\dcamm_env\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ASUS\anaconda3\envs\dcamm_env\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ASUS\anaconda3\envs\dcamm_env\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ASUS\anaconda3\envs\dcamm_env

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [11]:
race_pred.tail()

,name,race,api,black,hispanic,white
137,Negri,white,0.059991,0.016877,0.093277,0.768099
138,Madden,white,0.013587,0.084478,0.017829,0.943021
139,Woodlief,white,0.007713,0.044548,0.019787,0.569683
140,Manson,white,0.013241,0.154208,0.041408,0.881707
141,Hernandez,hispanic,0.013455,0.004020,0.924453,0.046696


In [12]:
names_with_races = race_pred.drop(['api', 'black', 'hispanic', 'white'], axis=1)
# black_count = race_pred['race'] == 'black'
# black_people = names_with_races[black_count]

In [13]:
names_with_races.to_csv('People_in_Boston_Globe_articles_2018.csv', index=True)